# Cost Optimization Analysis (Synthetic)

This notebook replicates a real-world **cost vs. sell price** analysis using a synthetic dataset.
Focus: margin distribution, outlier detection (IQR), and actionable insights by job_code and region.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
df = pd.read_csv('data/service_orders.csv', parse_dates=['order_date'])
df.head()

## 1. Basic sanity checks

In [ ]:
df.info()
df.describe(include='all').T.head(20)

## 2. Margin calculations & quick KPIs

In [ ]:
df['margin_amount'] = df['sell_price'] - df['base_cost']
df['margin_pct'] = np.where(df['sell_price']>0, df['margin_amount']/df['sell_price'], np.nan)

kpis = {
    'orders': len(df),
    'gross_revenue': df['sell_price'].sum(),
    'total_cost': df['base_cost'].sum(),
    'total_margin': df['margin_amount'].sum(),
    'median_margin_pct': df['margin_pct'].median()
}
kpis

## 3. Visualize margin distribution

In [ ]:
plt.figure()
plt.hist(df['margin_pct'].dropna(), bins=50)
plt.title('Margin % Distribution')
plt.xlabel('Margin %')
plt.ylabel('Count')
plt.show()

## 4. IQR outlier detection

In [ ]:
q1, q3 = df['margin_pct'].quantile([0.25, 0.75])
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
outliers = df[(df['margin_pct'] < lower) | (df['margin_pct'] > upper)]
outliers.shape, lower, upper

## 5. Slicing by job_code × region (Top loss-makers)

In [ ]:
grouped = (df
  .assign(loss_flag=lambda x: x['margin_amount'] < 0)
  .groupby(['job_code', 'region'], as_index=False)
  .agg(
      orders=('service_order_no', 'count'),
      avg_margin_pct=('margin_pct','mean'),
      loss_rate=('loss_flag','mean'),
      total_margin=('margin_amount','sum')
  )
  .sort_values(['total_margin'])
)
grouped.head(10)

## 6. Visualize top 10 loss-making pairs

In [ ]:
top_losses = grouped.nsmallest(10, 'total_margin')
plt.figure()
plt.barh(top_losses['job_code'] + ' | ' + top_losses['region'], top_losses['total_margin'])
plt.title('Top 10 Loss-making JobCode × Region (Total Margin)')
plt.xlabel('Total Margin (EUR)')
plt.ylabel('JobCode × Region')
plt.gca().invert_yaxis()
plt.show()

## 7. Warranty & discount impact

In [ ]:
w = (df.groupby('warranty_flag', as_index=False)
       .agg(avg_discount=('discount_pct','mean'),
            avg_margin_pct=('margin_pct','mean')))
w

## 8. Recommendations (fill in after inspecting results)

- Cap discounts on specific job_code × region pairs with high loss_rate.
- Review parts sourcing for codes 81120/82010 (high parts-cost multipliers).
- Consider adjusting list_price floors for warranty jobs.
